In [18]:
import pandas as pd
import numpy as np

# Explore the data


In [2]:
DATA_FOLDER = 'habits/'
answers_file = "questionnaire_answers.csv"
questions_file = 'questions_table.csv'

questionnaire_answers= pd.read_csv(DATA_FOLDER + answers_file, sep=',', error_bad_lines=False, encoding = "utf-8").dropna()
questionnaire_questions= pd.read_csv(DATA_FOLDER + questions_file, sep=',', error_bad_lines=False, encoding = "utf-8").dropna()

In [3]:
questionnaire_answers.head()

,questionnaire answer identifier,date,time,user identifier,question identifier,score of the answer,questionnaire identifier
0,151,2018-05-07,14:30:46,114,1,7,1
1,152,2018-05-07,14:30:50,114,2,7,1
2,153,2018-05-07,14:30:54,114,3,7,1
3,154,2018-05-07,14:30:57,114,4,7,1
4,155,2018-05-07,14:31:05,114,5,7,1


In [4]:
questionnaire_users_df = questionnaire_answers.groupby(['questionnaire identifier', 'user identifier']).size().reset_index()

questionnaire_users_df.head()

,questionnaire identifier,user identifier,0
0,1,114,12
1,1,316,1
2,1,321,12
3,1,325,12
4,1,327,12


In [5]:
questionnaire_questions.head()

,question identifier,text,questionnaire identifier
0,1,"En classe, je manque souvent des points import...",1
1,2,"Lorsque je lis un document pour ce cours, je p...",1
2,3,Lorsque je lis un document pour ce cours et qu...,1
3,4,Si les documents du cours sont difficiles à co...,1
4,5,Avant d’étudier un nouveau document du cours e...,1


In [6]:
questionnaire_questions_df = questionnaire_questions.groupby(['questionnaire identifier']).size().reset_index()

questionnaire_questions_df.head()

,questionnaire identifier,0
0,1,12
1,2,7
2,4,8
3,5,4
4,7,3


In [7]:
questionnaires_num = 25
questions_num = []
for i in range(questionnaires_num + 1):
    try:
        questions_num.append(int(questionnaire_questions_df.loc[questionnaire_questions_df['questionnaire identifier'] == i][0]))
    except:
        questions_num.append(0)

In [8]:
bad_indices =[]
for index, row in questionnaire_users_df.iterrows():
    if row[0] != questions_num[row['questionnaire identifier']]:
        bad_indices.append(index)
        
print(len(bad_indices))

66


In [9]:
questionnaire_users_cp = questionnaire_users_df.copy()
removed = 0
for x in bad_indices:
    try:
        questionnaire_users_cp = questionnaire_users_cp.drop(questionnaire_users_cp.index[x - removed])
        removed += 1
    except:
        print(str(x) + " not found")

In [10]:
questionnaire_users_cp.groupby(['questionnaire identifier']).size().sort_values(ascending=False)

questionnaire identifier
11    460
12    453
13    452
1     347
2     248
4     187
5     186
18    113
20    112
19    112
9     105
7     103
8      99
22     72
24     69
23     69
16     50
15     50
25      5
dtype: int64

In [12]:
questionnaire_answers.head()

,questionnaire answer identifier,date,time,user identifier,question identifier,score of the answer,questionnaire identifier
0,151,2018-05-07,14:30:46,114,1,7,1
1,152,2018-05-07,14:30:50,114,2,7,1
2,153,2018-05-07,14:30:54,114,3,7,1
3,154,2018-05-07,14:30:57,114,4,7,1
4,155,2018-05-07,14:31:05,114,5,7,1


In [45]:
def CronbachAlpha(itemscores):
    itemvars = itemscores.var(axis=1, ddof=1)
    tscores = itemscores.sum(axis=0)
    nitems = len(itemscores)

    return nitems / (nitems-1.) * (1 - itemvars.sum() / tscores.var(ddof=1))

In [69]:
def create_student_questions_matrix(df):
    students = pd.Series(df['user identifier'], name='A').unique().tolist()
    questions = pd.Series(df['question identifier'], name='A').unique().tolist()
    students_questions = np.zeros(shape=(len(questions), len(students)))
    
    for index, row in df.iterrows():
        s = students.index(row['user identifier'])
        q = questions.index(row['question identifier'])
        
        students_questions[q][s] = row['score of the answer']
    
    return students_questions

In [73]:
students_questions = create_student_questions_matrix(questionnaire_answers)
students_questions

array([[7., 3., 3., ..., 3., 6., 3.],
       [7., 5., 1., ..., 2., 4., 2.],
       [7., 6., 5., ..., 6., 7., 7.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [72]:
c = CronbachAlpha(students_questions)
c

0.9766371248207409

# Per questionnaire


In [52]:
qs = questionnaire_answers.groupby('questionnaire identifier')['questionnaire identifier'].unique()

In [74]:
questionnaires = pd.Series(questionnaire_answers['questionnaire identifier'], name='A').unique().tolist()
questionnaires

[1, 2, 4, 5, 7, 8, 11, 12, 13, 22, 23, 24, 18, 19, 20, 15, 16, 9, 25]

In [79]:
CronbachAlpha = []

for q in questionnaires:
    c = CronbachAlpha(create_student_questions_matrix(questionnaire_answers[questionnaire_answers['questionnaire identifier'] == q]))


TypeError: 'list' object is not callable

In [83]:
CronbachAlpha(create_student_questions_matrix(questionnaire_answers[questionnaire_answers['questionnaire identifier'] == 7]))

TypeError: 'list' object is not callable